In [229]:
# Imports
import numpy as np
import pandas as pd

from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode(connected=True)

import re
import string
import math
import random
import time

from IPython.display import display

import warnings  
warnings.filterwarnings('ignore')

In [83]:
# Settings

pd.set_option('display.max_columns', 100, 'display.width', 1024)
pd.options.mode.chained_assignment = None

DATA_PATH = '../Input/'


In [139]:
# Import Professionals

professionals = pd.read_csv(DATA_PATH + 'professionals.csv')
professionals = professionals.rename(columns={'professionals_location': 'location', 
                                              'professionals_id': 'professional_id',
                                              'professionals_industry': 'industry', 
                                              'professionals_headline': 'headline', 
                                              'professionals_date_joined': 'date_joined'})

professionals["date_joined"] = pd.to_datetime(professionals["date_joined"], infer_datetime_format=True)
professionals['headline'] = professionals['headline'].fillna('')
professionals['industry'] = professionals['industry'].fillna('')
professionals['location'] = professionals['location'].fillna('')



In [140]:
print(professionals.shape)
professionals.head()

(28152, 5)


,professional_id,location,industry,headline,date_joined
0,9ced4ce7519049c0944147afb75a8ce3,,,,2011-10-05 20:35:19
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,,,,2011-10-05 20:49:21
2,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",,,2011-10-18 17:31:26
3,977428d851b24183b223be0eb8619a8c,"Boston, Massachusetts",,,2011-11-09 20:39:29
4,e2d57e5041a44f489288397c9904c2b2,,,,2011-12-10 22:14:44


In [141]:
# Import Students

students = pd.read_csv(DATA_PATH + 'students.csv')
students = students.rename(columns={'students_location': 'location',
                                    'students_id': 'student_id',
                                    'students_date_joined': 'date_joined'})

students["date_joined"] = pd.to_datetime(students["date_joined"], infer_datetime_format=True)
students["location"] = students["location"].fillna("")



In [142]:
print(students.shape)
students.head()

(30971, 3)


,student_id,location,date_joined
0,12a89e96755a4dba83ff03e03043d9c0,,2011-12-16 14:19:24
1,e37a5990fe354c60be5e87376b08d5e3,,2011-12-27 03:02:44
2,12b402cceeda43dcb6e12ef9f2d221ea,,2012-01-01 05:00:00
3,a0f431fc79794edcb104f68ce55ab897,,2012-01-01 05:00:00
4,23aea4702d804bd88d1e9fb28074a1b4,,2012-01-01 05:00:00


In [23]:
students.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30971 entries, 12a89e96755a4dba83ff03e03043d9c0 to 1de9d7d0329e40e6996bbc92d13c6a00
Data columns (total 2 columns):
location       30971 non-null object
date_joined    30971 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 725.9+ KB


In [143]:
# Import Questions
questions = pd.read_csv(DATA_PATH + 'questions.csv', 
                        parse_dates=['questions_date_added'], 
                        infer_datetime_format=True)

questions = questions.rename(columns={'questions_author_id': 'author_id',
                                      'questions_id' :'question_id',
                                      'questions_date_added': 'date_added', 
                                      'questions_title': 'title', 
                                      'questions_body': 'body', 
                                      'questions_processed':'processed'})


In [144]:
print(questions.shape)
questions.head()

(23931, 5)


,question_id,author_id,date_added,title,body
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26,Teacher career question,What is a maths teacher? what is a ma...
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32,To become a specialist in business management...,i hear business management is a hard way to ge...
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...


In [146]:
# Import Answers
answers = pd.read_csv(DATA_PATH + 'answers.csv', 
                      parse_dates=['answers_date_added'], 
                      infer_datetime_format=True)

answers = answers.rename(columns={'answers_author_id':'author_id',
                                  'answers_id':'answer_id',
                                  'answers_question_id': 'question_id', 
                                  'answers_date_added': 'date_added', 
                                  'answers_body': 'body'})


In [147]:
print(answers.shape)
answers.head()

(51123, 5)


,answer_id,author_id,question_id,date_added,body
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14,<p>Hi!</p>\n<p>You are asking a very interesti...
1,ada720538c014e9b8a6dceed09385ee3,2aa47af241bf42a4b874c453f0381bd4,eb80205482e4424cad8f16bc25aa2d9c,2018-05-01 14:19:08,<p>Hi. I joined the Army after I attended coll...
2,eaa66ef919bc408ab5296237440e323f,cbd8f30613a849bf918aed5c010340be,eb80205482e4424cad8f16bc25aa2d9c,2018-05-02 02:41:02,"<p>Dear Priyanka,</p><p>Greetings! I have answ..."
3,1a6b3749d391486c9e371fbd1e605014,7e72a630c303442ba92ff00e8ea451df,4ec31632938a40b98909416bdd0decff,2017-05-10 19:00:47,<p>I work for a global company who values high...
4,5229c514000446d582050f89ebd4e184,17802d94699140b0a0d2995f30c034c6,2f6a9a99d9b24e5baa50d40d0ba50a75,2017-10-13 22:07:33,I agree with Denise. Every single job I've had...


In [148]:
# Import Tags
tags = pd.read_csv(DATA_PATH + 'tags.csv')
tags = tags.rename(columns={"tags_tag_id":"tag_id",'tags_tag_name': 'name'})

In [149]:
print(tags.shape)
tags.sample(10)

(16269, 2)


,tag_id,name
5258,33322,#athletics
6058,38896,careerdeveloping
7712,21197,dining
9871,11824,ibm
11229,38065,mastersprogram
5201,30132,asdfasdf
6218,19501,cg-lighting
10406,35426,i-will-do-it
12921,37007,privateaccounting
10999,38228,lowgpa


In [150]:
# Import Comments
comments = pd.read_csv(DATA_PATH + 'comments.csv', 
                       parse_dates = ["comments_date_added"], 
                       infer_datetime_format=True )

comments = comments.rename(columns={'comments_author_id': 'author_id',
                                    'comments_id':'comment_id',
                                    'comments_parent_content_id': 'parent_content_id', 
                                    'comments_date_added': 'date_added', 
                                    'comments_body': 'body' })


In [151]:
print(comments.shape)
comments.head()

(14966, 5)


,comment_id,author_id,parent_content_id,date_added,body
0,f30250d3c2ca489db1afa9b95d481e08,9fc88a7c3323466dbb35798264c7d497,b476f9c6d9cd4c50a7bacdd90edd015a,2019-01-31 23:39:40,"First, you speak to recruiters. They are train..."
1,ca9bfc4ba9464ea383a8b080301ad72c,de2415064b9b445c8717425ed70fd99a,ef4b6ae24d1f4c3b977731e8189c7fd7,2019-01-31 20:30:47,Most large universities offer study abroad pro...
2,c354f6e33956499aa8b03798a60e9386,6ed20605002a42b0b8e3d6ac97c50c7f,ca7a9d7a95df471c816db82ee758f57d,2019-01-31 18:44:04,"First, I want to put you at ease that the oppo..."
3,73a6223948714c5da6231937157e4cb7,d02f6d9faac24997a7003a59e5f34bd3,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 17:53:28,Your question submission was great! I just wan...
4,55a89a9061d44dd19569c45f90a22779,e78f75c543e84e1c94da1801d8560f65,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 14:51:53,Thank you. I'm new to this site. I'm sorry if ...


In [152]:
# Import School Memberships
school_memberships = pd.read_csv(DATA_PATH + 'school_memberships.csv')
school_memberships = school_memberships.rename(columns={'school_memberships_school_id': 'school_id', 
                                                        'school_memberships_user_id': 'user_id'})


In [153]:
print(school_memberships.shape)
school_memberships.head()

(5638, 2)


,school_id,user_id
0,197406,23dce13ca6164a73aec7a3cd56a4884d
1,197398,23dce13ca6164a73aec7a3cd56a4884d
2,199821,23dce13ca6164a73aec7a3cd56a4884d
3,186239,9c5803ae43ca4cf6b27ea85871625116
4,182063,9c5803ae43ca4cf6b27ea85871625116


In [154]:
# Groups Memberships
group_memberships = pd.read_csv(DATA_PATH + 'group_memberships.csv')
group_memberships = group_memberships.rename(columns={'group_memberships_group_id': 'group_id', 
                                                      'group_memberships_user_id': 'user_id'})


In [155]:
print(group_memberships.shape)
group_memberships.head()

(1038, 2)


,group_id,user_id
0,eabbdf4029734c848a9da20779637d03,9a5aead62c344207b2624dba90985dc5
1,eabbdf4029734c848a9da20779637d03,ea7122da1c7b4244a2184a4f9f944053
2,eabbdf4029734c848a9da20779637d03,cba603f34acb4a40b3ccb53fe6681b5d
3,eabbdf4029734c848a9da20779637d03,fa9a126e63714641ae0145557a390cab
4,eabbdf4029734c848a9da20779637d03,299da113c5d1420ab525106c242c9429


In [156]:
# Emails
emails = pd.read_csv(DATA_PATH + 'emails.csv', 
                     parse_dates = ["emails_date_sent"], 
                     infer_datetime_format=True)

emails = emails.rename(columns={'emails_recipient_id':'recipient_id',
                                "emails_id":"email_id",
                                'emails_date_sent': 'date_sent', 
                                'emails_frequency_level': 'frequency_level'})


In [157]:
print(emails.shape)
emails.head()

(1850101, 4)


,email_id,recipient_id,date_sent,frequency_level
0,2337714,0c673e046d824ec0ad0ebe012a0673e4,2018-12-07 01:05:40,email_notification_daily
1,2336077,0c673e046d824ec0ad0ebe012a0673e4,2018-12-06 01:14:15,email_notification_daily
2,2314660,0c673e046d824ec0ad0ebe012a0673e4,2018-11-17 00:38:27,email_notification_daily
3,2312639,0c673e046d824ec0ad0ebe012a0673e4,2018-11-16 00:32:19,email_notification_daily
4,2299700,0c673e046d824ec0ad0ebe012a0673e4,2018-11-08 00:16:40,email_notification_daily


In [158]:
# Questions-related stats
tag_questions = pd.read_csv(DATA_PATH + 'tag_questions.csv',)
tag_questions = tag_questions.rename(columns={'tag_questions_tag_id': 'tag_id', 
                                              'tag_questions_question_id': 'question_id'})

In [159]:
print(tag_questions.shape)
tag_questions.head()

(76553, 2)


,tag_id,question_id
0,28930,cb43ebee01364c68ac61d347a393ae39
1,28930,47f55e85ce944242a5a347ab85a8ffb4
2,28930,ccc30a033a0f4dfdb2eb987012f25792
3,28930,e30b274e48d741f7bf50eb5e7171a3c0
4,28930,3d22742052df4989b311b4195cbb0f1a


In [80]:
count_question_tags = (tag_questions.groupby('question_id')
                       .count()
                       .rename(columns={'tag_id': 'count_tags'})
                       .sort_values('count_tags', ascending=False))

In [81]:
count_question_tags.head()

,count_tags
question_id,
e79bf4570af646d5892cf42b031c2a52,54
2ea130631ba34b4181c5fd85816504cf,53
e1860d4512b746a19270e5675efb7b44,52
164522e7595649729deebf48cad87e1b,47
0da559c9445e4ad59a98611e164c3ddd,44


In [79]:
print('\nInteresting statistics: ')
print(f'- {(answers["question_id"].nunique()/len(questions))*100:.2f} % of the questions have at least 1 answer.')
print(f'\n- {(len(count_question_tags)/len(questions))*100:.2f}% of questions are tagged by at least {count_question_tags["count_tags"].tail(1).values[0]} tag.')
print(f'- Mean of tags per question: {count_question_tags["count_tags"].mean():.2f} tags per question.')


Interesting statistics: 
- 96.57 % of the questions have at least 1 answer.

- 97.31% of questions are tagged by at least 1 tag.
- Mean of tags per question: 3.29 tags per question.


In [88]:
tag_users = pd.read_csv(DATA_PATH + 'tag_users.csv',)
tag_users = tag_users.rename(columns={'tag_users_tag_id': 'tag_id', 
                                      'tag_users_user_id': 'user_id'})


In [89]:
print(tag_users.shape)
tag_users.head()

(136663, 2)


,tag_id,user_id
0,593,c72ab38e073246e88da7e9a4ec7a4472
1,1642,8db519781ec24f2e8bdc67c2ac53f614
2,638,042d2184ee3e4e548fc3589baaa69caf
3,11093,c660bd0dc1b34224be78a58aa5a84a63
4,21539,8ce1dca4e94240239e4385ed22ef43ce


In [90]:
users_who_follow_tags = list(tag_users['user_id'].unique())
nbr_pros_tags = len(professionals[professionals.index.isin(users_who_follow_tags)])
nbr_students_tags = len(students[students.index.isin(users_who_follow_tags)])
print(f'\n- {(nbr_pros_tags / len(professionals))*100:.2f} % of the professionals follow at least 1 Tag ({nbr_pros_tags}).')
print(f'- {(nbr_students_tags / len(students))*100:.2f} % of the students follow at least 1 Tag ({nbr_students_tags}).')



- 90.91 % of the professionals follow at least 1 Tag (25594).
- 14.88 % of the students follow at least 1 Tag (4608).


In [161]:
question_scores = pd.read_csv(DATA_PATH + 'question_scores.csv')
question_scores = question_scores.rename(columns = {"id":"question_id", "score":"question_score"})
question_scores.head()


,question_id,question_score
0,38436aadef3d4b608ad089cf53ab0fe7,5
1,edb8c179c5d64c9cb812a59a32045f55,4
2,333464d7484b43e3866e86096bc4ddb9,6
3,4b995e60b99d4ee18346e893e007cb8f,6
4,f6b9ca94aed04ba28256492708e74f60,6


In [163]:
nbr_questions_with_hearts = len(question_scores[question_scores['question_score'] > 0]['question_id'])
print(f'\n- {(nbr_questions_with_hearts/len(questions))*100:.2f} % of questions were upvoted ({nbr_questions_with_hearts} out of {len(question_scores)} ).')



- 96.93 % of questions were upvoted (23196 out of 23928 ).


In [164]:
question_scores.sort_values("question_score", ascending=False).head()

,question_id,question_score
18158,25f7d8ba4bb748b8bbb11683c0aafe18,125
18159,95ddad4e5e9049c483f5c7ebea940bbe,123
18157,1d37482ddd534627bfcb1826c729cb0a,122
18155,0bfd2853655d4daaa13333d42b2cbeed,122
18156,cc83f19c73694fc6ac90e07261a8af6f,121


In [168]:
answer_scores = pd.read_csv(DATA_PATH + 'answer_scores.csv')
answer_scores = answer_scores.rename(columns = {"id":"answer_id", "score":"answer_score"})
answer_scores.head()


,answer_id,answer_score
0,7b2bb0fc0d384e298cffa6afde9cf6ab,1
1,7640a6e5d5224c8681cc58de860858f4,5
2,3ce32e236fa9435183b2180fb213375c,2
3,fa30fe4c016043e382c441a7ef743bfb,0
4,71229eb293314c8a9e545057ecc32c93,2


In [126]:
answer_scores.shape

(51138, 2)

In [167]:
nbr_answers_with_hearts = len(answer_scores[answer_scores['answer_score'] > 0]['answer_id'])
print(f'- {(nbr_answers_with_hearts/len(questions))*100:.2f} % of answers were upvoted ({nbr_answers_with_hearts}).')

- 57.82 % of answers were upvoted (13837).


In [166]:
answer_scores.sort_values("answer_score", ascending=False).head()

,answer_id,answer_score
53,c643a5cc6a4f4f02bc91bd663cf33577,30
6836,f840b044ffc547bcbbe0dac9e6d69e6d,25
98,4570cb0d84d6478bad46e9b1741e4ec9,22
1061,0c4067ad61f4486ebbae927ea0983175,20
242,281c2557f0154dde984707e40100b8b4,19


In [130]:
matches = pd.read_csv(DATA_PATH + "matches.csv")
matches = matches.rename(columns = {"matches_email_id" : "email_id", "matches_question_id":"question_id"})

In [132]:
print(matches.shape)
matches.head()

(4316275, 2)


,email_id,question_id
0,1721939,332a511f1569444485cf7a7a556a5e54
1,1665388,332a511f1569444485cf7a7a556a5e54
2,1636634,332a511f1569444485cf7a7a556a5e54
3,1635498,332a511f1569444485cf7a7a556a5e54
4,1620298,332a511f1569444485cf7a7a556a5e54


In [135]:
groups = pd.read_csv(DATA_PATH + "groups.csv")
groups = groups.rename(columns = {"groups_id":"group_id", "groups_group_type":"group_type"})

In [137]:
groups.head()

,group_id,group_type
0,eabbdf4029734c848a9da20779637d03,youth program
1,7080bf8dcf78463bb03e6863887fd715,youth program
2,bc6fc50a2b444efc8ec47111b290ffb8,youth program
3,37f002e8d5e442ca8e36e972eaa55882,youth program
4,52419ff84d4b47bebd0b0a6c1263c296,youth program


In [171]:
#####################################################
print('Important numbers:')
print('\nThere are:')
print(f'- {len(students)} Students.', end="\t")
print(f'- {len(professionals)} Professionals.')
print(f'- {len(questions)} Questions.', end="\t")
print(f'- {len(answers)} Answers.')
print(f'- {len(tags)} Tags.', end="\t\t")
print(f'- {len(comments)} Comments.')
print(f'- {school_memberships["school_id"].nunique()} Schools.', end="\t\t")
print(f'- {len(pd.read_csv("../input/groups.csv"))} Groups.')
print(f'- {len(emails)} Emails were sent.')
#####################################################

# School/Group Related Stats

def is_student(user_id):
    if user_id in students.student_id.values:
        return 1
    elif user_id in professionals.professional_id.values:
        return 0
    else:
        raise ValueError('User ID not student & not professional')

school_memberships['is_student'] = school_memberships['user_id'].apply(is_student)
school_memberships['is_student'] = school_memberships['is_student'].astype(int)
count_students_professionals = school_memberships.groupby('is_student').count()[['school_id']].rename(columns={'school_id':'count'})
print(f'\n- Only {count_students_professionals.loc[1].values[0]/len(students):.2f} % of the students are members of schools ({count_students_professionals.loc[1].values[0]}).')
print(f'- Only {count_students_professionals.loc[0].values[0]/len(professionals):.2f} % of the professionals are members of schools ({count_students_professionals.loc[0].values[0]}).')

group_memberships['is_student'] = group_memberships['user_id'].apply(is_student)
group_memberships['is_student'] = group_memberships['is_student'].astype(int)
count_students_professionals = group_memberships.groupby('is_student').count()[['group_id']].rename(columns={'group_id':'count'})
print(f'\n- Only {count_students_professionals.loc[1].values[0]/len(students):.2f} % of the students are members of groups ({count_students_professionals.loc[1].values[0]}).')
print(f'- Only {count_students_professionals.loc[0].values[0]/len(professionals):.2f} % of the professionals are members of groups ({count_students_professionals.loc[0].values[0]}).')


print('')

Important numbers:

There are:
- 30971 Students.	- 28152 Professionals.
- 23931 Questions.	- 51123 Answers.
- 16269 Tags.		- 14966 Comments.
- 2706 Schools.		- 49 Groups.
- 1850101 Emails were sent.

- Only 0.04 % of the students are members of schools (1355).
- Only 0.15 % of the professionals are members of schools (4283).

- Only 0.01 % of the students are members of groups (311).
- Only 0.03 % of the professionals are members of groups (727).



In [188]:
pros_with_50more_answer = pros_with_answer[(pros_with_answer.values>50)].count()
pros_with_50more_answer

95

In [191]:
# Professionals Activity
pros_without_answer = len(professionals) - answers['author_id'].nunique()

pros_with_answer = answers['author_id'].value_counts().sort_values()

pros_with_only_one_answer = pros_with_answer[pros_with_answer.values==1].count()
pros_with_2to5_answer = pros_with_answer[(pros_with_answer.values>1) & (pros_with_answer.values<6)].count()
pros_with_5to10_answer = pros_with_answer[(pros_with_answer.values>5) & (pros_with_answer.values<11)].count()
pros_with_10to20_answer = pros_with_answer[(pros_with_answer.values>10) & (pros_with_answer.values<21)].count()
pros_with_20to50_answer = pros_with_answer[(pros_with_answer.values>20) & (pros_with_answer.values<51)].count()
pros_with_50more_answer = pros_with_answer[(pros_with_answer.values>50)].count()


In [259]:
import plotly.plotly as py

fig = {
  "data": [
    {
      "values": [pros_without_answer,
                 pros_with_only_one_answer,
                 pros_with_2to5_answer, 
                 pros_with_5to10_answer, 
                 pros_with_10to20_answer, 
                 pros_with_20to50_answer, 
                 pros_with_50more_answer],
      "labels": [
        "Professionals without answers",
        "Professionals with only one answer",
        "Professionals with 2 to 5 answers",
        "Professionals with 5 to 10 answers",
        "Professionals with 10 to 20 answers",
        "Professionals with 20 to 50 answers",
        "Professionals with more than 20 answers"
      ],
      "domain": {"column": 0},
      "name": "Professional activity",
      "hoverinfo":"label+percent+name",
      "hole": .6,
      "type": "pie"
    }],
  "layout": {
        "title":"Professional contribution in answering questions",
        "grid": {"rows": 1, "columns": 1},
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": " ",
                "x": 0.50,
                "y": 0.5
            }
        ]
    }
}
iplot(fig, filename='donut')

In [238]:
professionals["year"] = professionals["date_joined"].dt.year
professionals.shape

(28152, 6)

In [228]:
professionals_joining_over_time = pd.DataFrame(professionals
                                           .groupby("year")["professional_id"]
                                           .count())
professionals_joining_over_time = professionals_joining_over_time.rename(columns = {"professional_id":"number of professionals added"})
professionals_joining_over_time = professionals_joining_over_time.reset_index()
professionals_joining_over_time

,year,number of professionals added
0,2011,57
1,2012,222
2,2013,429
3,2014,987
4,2015,1540
5,2016,5618
6,2017,6322
7,2018,11572
8,2019,1405


In [240]:
len(professionals_joining_over_time)

9

In [244]:
total_number_of_pros = []
sum = 0
for i in range (len(professionals_joining_over_time)):
    sum+=professionals_joining_over_time["number of professionals added"].values[i]
    total_number_of_pros.append(sum)

In [246]:
professionals_joining_over_time["total_number_of_pros"] = total_number_of_pros

In [247]:
professionals_joining_over_time

,year,number of professionals added,total_number_of_pros
0,2011,57,57
1,2012,222,279
2,2013,429,708
3,2014,987,1695
4,2015,1540,3235
5,2016,5618,8853
6,2017,6322,15175
7,2018,11572,26747
8,2019,1405,28152


In [227]:
answers["year"] = answers["date_added"].dt.year
Active_professional_over_time =pd.DataFrame(answers.groupby("year")["author_id"].nunique()) 
Active_professional_over_time = Active_professional_over_time.rename(columns = {"author_id":"number of active professionals"})
Active_professional_over_time = Active_professional_over_time.reset_index()
Active_professional_over_time

,year,number of active professionals
0,2011,23
1,2012,147
2,2013,180
3,2014,581
4,2015,797
5,2016,2679
6,2017,3130
7,2018,4094
8,2019,413


In [249]:
pro = professionals_joining_over_time.merge(Active_professional_over_time)
pro["number of non-active professionals"] = pro["total_number_of_pros"]-pro["number of active professionals"]
pro

,year,number of professionals added,total_number_of_pros,number of active professionals,number of non-active professionals
0,2011,57,57,23,34
1,2012,222,279,147,132
2,2013,429,708,180,528
3,2014,987,1695,581,1114
4,2015,1540,3235,797,2438
5,2016,5618,8853,2679,6174
6,2017,6322,15175,3130,12045
7,2018,11572,26747,4094,22653
8,2019,1405,28152,413,27739


In [262]:

trace1 = go.Bar(
    x=pro["year"],
    y=pro["number of non-active professionals"],
    name='Inactive',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)
trace2 = go.Bar(
    x=pro["year"],
    y=pro["number of active professionals"],
    name='Active',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)
data = [trace1, trace2]
layout = go.Layout(
    title='Professionals Activity over Time',
    xaxis=dict(
        title='Year',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Number of Professionals',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    font=dict(
            family='sans-serif',
            size=16,
            color='#000'
        ),
    barmode='stack',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='style-bar')


In [257]:
professionals['month_year'] = professionals['date_joined'].dt.to_period('M')
students['month_year'] = students['date_joined'].dt.to_period('M')
['month_year'] = professionals['date_joined'].dt.to_period('M')
professionals['month_year'] = professionals['date_joined'].dt.to_period('M')


In [258]:
professionals.head()

,professional_id,location,industry,headline,date_joined,year,month_year
0,9ced4ce7519049c0944147afb75a8ce3,,,,2011-10-05 20:35:19,2011,2011-10
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,,,,2011-10-05 20:49:21,2011,2011-10
2,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",,,2011-10-18 17:31:26,2011,2011-10
3,977428d851b24183b223be0eb8619a8c,"Boston, Massachusetts",,,2011-11-09 20:39:29,2011,2011-11
4,e2d57e5041a44f489288397c9904c2b2,,,,2011-12-10 22:14:44,2011,2011-12


In [260]:
import plotly.io as pio
